In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import csv
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM

#Encoder-Decoder Model
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer
from transformers import Trainer
from evaluate import load
#Training

from transformers import TrainingArguments
from transformers import  Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

from datasets import Dataset
import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
# bertscore = load("bertscore") 
tokenizer =  AutoTokenizer.from_pretrained("t5-small",cahe_dir='/ssd_scratch/cvit/aparna/t5_small')
model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/t5_simplification_final")

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_labels)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [3]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*10)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 34%|███▎      | 3356/10000 [48:48<1:33:36,  1.18it/s]

In [ ]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*1)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*1)
    eval_arg = [inpu,cands,lab]


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 10000/10000 [1:39:52<00:00,  1.67it/s] 


10000 10000 10000
10000 10000 10000
3
(0.04406381827914688, 36.21581566046019)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*1)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*1)
    eval_arg = [inpu,cands,lab]


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 10000/10000 [1:32:38<00:00,  1.80it/s] 


10000 10000 10000
10000 10000 10000
3
(0.0011584308218406823, 50.138435269970884)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/t5_simplification_final")
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    print(text)
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
    print("====================================")
   
    for i, beam in enumerate(beam_outputs):
        print("{} {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        


  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


However , this option is susceptible to certain chosen-plaintext or known-plaintext attacks , and thus it is designated by NIST to have only 80 bits of security .


  0%|          | 1/10000 [00:01<4:48:32,  1.73s/it]

0 However, this option is susceptible to certain chosen-plaintext or known, and therefore it is designated by NIST to have only 80 bits of security.
They were the founders and the ruling dynasty of the kingdom of Macedon from about 700 to 310 BC .


  0%|          | 2/10000 [00:03<4:43:40,  1.70s/it]

...

0 He was born in New York City to Cuban parents Ne  stor Tulio Carbonell Cortina
Since 2001 , she has also served on the UN Secretary General 's Advisory Board on Disarmament Affairs ( 2001 – 2005 ) and continued to do so while holding the post of Pakistan 's High Commissioner to the United Kingdom ( 2003 – 2008 ) .


  2%|▏         | 166/10000 [05:13<7:14:37,  2.65s/it]

0 She has also served on the UN Secretary General's Advisory Board on Disarmament Affairs ( 2001 – 2005 ) and continued to do so while holding the post of Pakistan '
He died after a brief hospitalization in 2014 .


  2%|▏         | 166/10000 [05:14<5:10:21,  1.89s/it]


KeyboardInterrupt: 